In [8]:
import pandas as pd
import requests
import bs4
from bs4 import BeautifulSoup  
import json
import csv
import numpy as np
import matplotlib.pyplot as plt      
from datetime import datetime
from datetime import timedelta
import math


In [9]:
df = pd.read_csv("Data/Real_Estate_TLV_NADLAN.csv",index_col=0)
df2 = pd.read_csv("Data/Real_Estate_TLV_Step_3.csv",index_col=0)

df.rename(columns={'DISPLAYADRESS':'ADDRESS'}, inplace=True)
df2.rename(columns={'DEALDATESTRING':'DEALDATE'}, inplace=True)



In [10]:
def drop_col_from_df(df, cols):
    return df.drop(columns=cols, axis=1)

def convert_price(df):
    df['DEALAMOUNT'] = df['DEALAMOUNT'].str.replace(',', '').astype(int)
    return df

cols_to_drop = ['DEALNATUREDESCRIPTION',"DEALDATETIME","FULLADRESS","GUSH","PROJECTNAME","YEARBUILT","KEYVALUE" ,'TREND_IS_NEGATIVE','TREND_FORMAT','NEWPROJECTTEXT','TYPE','POLYGON_ID']

df = drop_col_from_df(df,cols_to_drop)
df = df.dropna().reset_index(drop=True)
df = convert_price(df)

In [11]:
def merge_dataframes(df1, df2):
    df1['DEALDATE'] = pd.to_datetime(df1['DEALDATE']).dt.strftime('%d.%m.%Y')
    df2['DEALDATE'] = pd.to_datetime(df2['DEALDATE']).dt.strftime('%d.%m.%Y')
    new_df = df1.merge(df2, on=['DEALDATE', 'ADDRESS'], how='outer')
    new_df.dropna()
    return new_df
 

def drop_rows_by_price(df):
    #delete all the rows that contain the value in the parameter and return new dataframe
    return df[(df["DEALAMOUNT"] >= 600000) & (df["DEALAMOUNT"] <= 20000000)].reset_index(drop=True)

def drop_rows_by_value_size(df):
    #delete all the rows that contain the value in the parameter and return new dataframe
    return df[(df["ASSETMETER"] >= 30) & (df["ASSETMETER"] <= 350)].reset_index(drop=True)


drop_col = ['FLOORNO','ASSETROOMNUM','centroid','ADDRESS','STREENNAME','DEALNATURE','DEALS_DEALAMOUNT']

merged_df = merge_dataframes(df, df2)
merged_df = drop_rows_by_value_size(merged_df)
merged_df = drop_rows_by_price(merged_df)
merged_df = drop_col_from_df(merged_df,drop_col)


In [12]:
def drop_non_numeric_cols(df):
    cols = df.select_dtypes(include=['int64', 'float64','int32']).columns
    cols = cols.drop('PRICE_PER_METER')
    return df[cols].dropna()

def get_year(df):
    df['DEALDATE'] = pd.to_datetime(df['DEALDATE']).dt.year
    df.rename(columns = {'DEALDATE':'DATE'}, inplace = True)
    df['DATE'] = df['DATE'].astype(int)
    return df

def NEIGHBORHOOD_to_numeric(df):
    t = df['NEIGHBORHOOD'].unique()
    df['NEIGHBORHOOD'].replace(t.tolist(),[i for i in range(len(t))], inplace=True)
    return df



In [13]:
merged_df = get_year(merged_df)
# merged_df.to_csv('Data/Real_Estate_TLV_Full_DATA.csv') 
# merged_df = NEIGHBORHOOD_to_numeric(merged_df)
# merged_df = drop_non_numeric_cols(merged_df)

merged_df['BUILDINGYEAR'] = merged_df['BUILDINGYEAR'].astype(int)


merged_df.to_csv('Data/Real_Estate_TLV_Numric_Data.csv') 


,DATE,DEALAMOUNT,BUILDINGYEAR,BUILDINGFLOORS,NEIGHBORHOOD,PRICE_PER_METER,ASSETTYPE,ROOMNUM,FLOOR,ASSETMETER,long,lat,Distance_From_Sea,AVG_SALARY
0,2022,2019000.0,1970,4.0,נוה חן,28842.0,דירה בבית קומות,4.0,5.0,70.0,661887.8100,182342.5000,5108.0,5574.0
1,2022,3410000.0,1940,2.0,קרית שלום,36276.0,דירה בבית קומות,4.0,2.0,94.0,661229.1900,178777.2000,1936.0,4863.0
2,2022,3900000.0,2019,4.0,כפיר,28467.0,דירה בבית קומות,5.0,22.0,137.0,661876.0131,181306.8267,4130.0,5655.0
3,2022,3900000.0,2019,4.0,כפיר,28467.0,דירה בבית קומות,5.0,22.0,137.0,661853.4802,181259.9201,4092.0,5655.0
4,2022,3200000.0,1950,1.0,קרית שלום,43243.0,דירה בבית קומות,3.5,1.0,74.0,661382.8800,179405.0900,2483.0,4863.0
